In [ ]:
# Install Package

!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 15.2 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.1
    Uninstalling graphviz-0.20.1:
      Successfully uninstalled graphviz-0.20.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 344.5/344.5 kB 10.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp39-cp39-linux_x86_64.whl size=680541 sha256=58dbd5821ad6ea076133baf6dbcd739d21e8b4a00c18783d7569c5c10930576b
  Stored in directory: /root/.cache/pip/wheels/47/17/70/b257bc53879a458c4bfcc900e89271aa8b4f19366a54bd2455
Successfully built gluonnlp
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np
import os
import progressbar
import time
# Import Library

import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
from tqdm import tqdm, tqdm_notebook
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
# Set GPU environment
device = torch.device("cuda:0")
# Install KoBert
bertmodel, vocab = get_pytorch_kobert_model()

# Set BERT Tokenizer

class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

## Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

# KoBERT Modeling.

class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=5, ##Categories. 0 = Very Negative, 1 = Negative, 2 = Neutral, 3 = Positive, 4 = Very Positive.
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
PATH = '/content/drive/MyDrive/Project/LiveCommerse/OpenAI/KoBERT/'
model = torch.load(PATH + 'KoBERT_GPT-turn.pt')  # 전체 모델을 통째로 불러옴, 클래스 선언 필수
model.load_state_dict(torch.load(PATH + 'model_state_dict.pt')) 

In [ ]:
main_path = '/content/drive/MyDrive/Project/LiveCommerse/'

In [ ]:
Path_Beauty = '/content/drive/MyDrive/Project/LiveCommerse/FUCKUGOOGLE/Beauty/Video'
Path_Food = '/content/drive/MyDrive/Project/LiveCommerse/FUCKUGOOGLE/Food/Video'
Path_Life = '/content/drive/MyDrive/Project/LiveCommerse/FUCKUGOOGLE/Life/Video'
Path_Fashion = '/content/drive/MyDrive/Project/LiveCommerse/FUCKUGOOGLE/Fashion/Video'
Beauty_list = os.listdir(Path_Beauty)
Food_list = os.listdir(Path_Food)
Life_list = os.listdir(Path_Life)
Fashion_list = os.listdir(Path_Fashion)
path_list = [Path_Beauty, Path_Food, Path_Life, Path_Fashion]

In [ ]:
# 학습된 KoBERT 불러온 뒤 예측.

def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]
    
    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)
        prediction = out.cpu().detach().numpy().argmax()

    return prediction

In [ ]:
map_dict = dict({4: 2, 3: 1, 2: 0, 1: -1, 0: -2})
for path in path_list[2:3]:
  total_df = pd.DataFrame()
  video_list = os.listdir(path)
  for a, video in enumerate(video_list):
    video_num = video.split('_')[1]
    video_num = video_num.split('.')[0]
    video_path = (path + '/' + video)
    try:
      video_df = pd.read_csv(video_path)
      bar = progressbar.ProgressBar(maxval = len(video_df)).start()
      for i in range(len(video_df)):
        bar.update(i)
        video_df.loc[i, 'KoBERT_Sentiment_2'] = predict(video_df.loc[i, 'chat'])
        video_df['KoBERT_Sentiment_1'] = video_df['KoBERT_Sentiment_2'].map(lambda x: map_dict.get(x))
        video_df.to_csv(video_path)
      print(f'Done, {video_path}')
      bar.finish()
    except:
      print(f'wrong, {video_path}')